<a href="https://colab.research.google.com/github/kyunghyuncho/ammi-2019-nlp/blob/master/01-day-LM/ngram_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# N-gram Language Modeling

In [1]:
# # run if you dont have it installed
# !pip install more_itertools
# !pip install spacy

In [2]:
import spacy
import torch
from torch.utils.data import Dataset, DataLoader
import random
import numpy
import itertools
from operator import itemgetter 
from glob import glob
from tqdm import tqdm_notebook, tqdm
from collections import Counter
import torch.nn as nn
import torch.nn.functional as F
import string
import re
import more_itertools as mit  # not built-in package
import torch
import torchtext
import torchtext.data as data
from torchtext import vocab
from collections import Counter
import re
from torchtext.data import TabularDataset 

In [3]:
torch.manual_seed(1)


In [4]:
import torch
import torchtext
import torchtext.data as data
from torchtext import vocab
from collections import Counter
import re
from torchtext.data import TabularDataset 

class AmazonReviewsDataset(TabularDataset):
    
    urls = [
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Books_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Movies_and_TV_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_CDs_and_Vinyl_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Clothing_Shoes_and_Jewelry_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Home_and_Kitchen_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Kindle_Store_5.json.gz',
           'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Sports_and_Outdoors_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Health_and_Personal_Care_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Toys_and_Games_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Tools_and_Home_Improvement_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Beauty_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Apps_for_Android_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Pet_Supplies_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Automotive_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Grocery_and_Gourmet_Food_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Patio_Lawn_and_Garden_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Baby_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Digital_Music_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz',
#             'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Amazon_Instant_Video_5.json.gz',
        ]
    name='amazonreviews'
    dirname='processed'

In [5]:
download_done = AmazonReviewsDataset.download(root='data/', check=True)

ReadError: invalid header

In [6]:
RETOK = re.compile(r'\w+|[^\w\s]|\n', re.UNICODE)

def tokenize(s):
    return RETOK.findall(s)

text_field = data.Field(sequential=True, tokenize=tokenize, include_lengths=True, use_vocab=True, lower=True, batch_first=True)

In [7]:
dataset = AmazonReviewsDataset(path='/home/roberta/ammi-2019-nlp/01-day-LM/data/amazonreviews/reviews_Sports_and_Outdoors_5.json', format='json', fields={'reviewText': ('reviewText', text_field), 'summary': ('summary', text_field)})

In [8]:
# lets check it
# lets use fstrings btw
print(f'Number of samples : {len(dataset.examples)}')

for ex in dataset.examples:
    print(f'Review: \n {ex.reviewText} \n\n Summary: \n {ex.summary}')
    break

Number of samples : 296337
Review: 
 ['this', 'came', 'in', 'on', 'time', 'and', 'i', 'am', 'veru', 'happy', 'with', 'it', ',', 'i', 'haved', 'used', 'it', 'already', 'and', 'it', 'makes', 'taking', 'out', 'the', 'pins', 'in', 'my', 'glock', '32', 'very', 'easy'] 

 Summary: 
 ['woks', 'very', 'good']


In [16]:
# convert the dataset to a list of strings
# each string represents a review
all_reviews = []
for ex in dataset.examples:
    all_reviews.append(ex.reviewText)
len(all_reviews)


296337

In [17]:
all_reviews[5]

['needed',
 'this',
 'tool',
 'to',
 'really',
 'break',
 'down',
 'my',
 'g22',
 ',',
 'and',
 'it',
 'works',
 'perfectly',
 'for',
 'that',
 '.',
 'no',
 'difference',
 'from',
 'oem',
 'i',
 'suspect',
 '.',
 'however',
 ',',
 'had',
 'an',
 'added',
 'bonus',
 'when',
 'realizing',
 'that',
 'i',
 'needed',
 'a',
 'punch',
 'to',
 'properly',
 'disassemble',
 'the',
 'bolt',
 'carrier',
 'in',
 'my',
 'ar',
 'for',
 'a',
 'thorough',
 'cleaning',
 ',',
 'and',
 'this',
 'tool',
 'worked',
 'perfectly',
 'to',
 'push',
 'the',
 'carrier',
 "'",
 's',
 'retaining',
 'pin',
 'out',
 'and',
 'then',
 'back',
 'into',
 'place',
 'after',
 'the',
 'job',
 'was',
 'completed',
 '.',
 'excellent',
 '!',
 'it',
 'is',
 'now',
 'in',
 'my',
 'range',
 'bag',
 'for',
 'safe',
 'keeping',
 '.']

In [18]:
_tqdm = tqdm_notebook  # prolly you need jupyter widget for this, change for tqdm for simple tqdm

NUM_SENTENCES = len(all_reviews)
NUM_SENTENCES_TRAIN = int(0.8*NUM_SENTENCES)
NUM_SENTENCES_TEST = int(0.1*NUM_SENTENCES)
NUM_SENTENCES_VALID = NUM_SENTENCES - NUM_SENTENCES_TRAIN - NUM_SENTENCES_TEST

train_reviews = all_reviews[:NUM_SENTENCES_TRAIN]
test_reviews = all_reviews[NUM_SENTENCES_TRAIN:NUM_SENTENCES_TRAIN+NUM_SENTENCES_TEST]
valid_reviews = all_reviews[NUM_SENTENCES_TRAIN+NUM_SENTENCES_TEST:NUM_SENTENCES_TRAIN+NUM_SENTENCES_TEST+NUM_SENTENCES_VALID]



In [19]:
NUM_SENTENCES_TRAIN, NUM_SENTENCES_TEST, NUM_SENTENCES_VALID, NUM_SENTENCES

(237069, 29633, 29635, 296337)

In [20]:
len(train_reviews), len(test_reviews), len(valid_reviews), len(train_reviews) + len(test_reviews) + len(valid_reviews) 

(237069, 29633, 29635, 296337)

In [21]:
# Load English tokenizer, tagger, parser, NER and word vectors
# !python -m spacy download en_core_web_sm
tokenizer = spacy.load('en_core_web_sm')
punctuations = string.punctuation
TAG_RE = re.compile(r'<[^>]+>') # get rid off HTML tags from the data

def remove_tags(text):
    return TAG_RE.sub('', text)

def lower_case_remove_punc(parsed):
    return [token.text.lower() for token in parsed if (token.text not in punctuations)] #and (token.is_stop is False)]

def tokenize_dataset(dataset):
    token_dataset = []
    # we are keeping track of all tokens in dataset 
    # in order to create vocabulary later
    all_tokens = []

    for sample in tqdm(tokenizer.pipe(dataset, disable=['parser', 'tagger', 'ner'], batch_size=512, n_threads=1)):
        tokens = lower_case_remove_punc(sample)
        token_dataset.append(tokens)
        all_tokens += tokens
        
    return token_dataset, all_tokens

In [22]:
punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [23]:
TAG_RE

re.compile(r'<[^>]+>', re.UNICODE)

In [24]:
train_reviews[0]

['this',
 'came',
 'in',
 'on',
 'time',
 'and',
 'i',
 'am',
 'veru',
 'happy',
 'with',
 'it',
 ',',
 'i',
 'haved',
 'used',
 'it',
 'already',
 'and',
 'it',
 'makes',
 'taking',
 'out',
 'the',
 'pins',
 'in',
 'my',
 'glock',
 '32',
 'very',
 'easy']

In [ ]:
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager


20070953it [12:58, 28924.68it/s]

In [32]:
train_reviews_flatten = [item for sublist in train_reviews for item in sublist]
test_reviews_flatten = [item for sublist in test_reviews for item in sublist]
valid_reviews_flatten = [item for sublist in valid_reviews for item in sublist]



In [ ]:
train_reviews_flatten[1]

'came'

In [ ]:
train_reviews_toked, num_train = tokenize_dataset(train_reviews_flatten)
test_reviews_toked, num_test = tokenize_dataset(test_reviews_flatten)
valid_reviews_toked, num_valid = tokenize_dataset(valid_reviews_flatten)

voc = list(set(num_train + num_test + num_valid))
print('Word vocabulary size: {} words'.format(len(voc)))
             
# TODO: this takes a really long time !! why?
def find_ngrams(input_list, n):
    result_list = []
    for l in input_list:
        result_list.append(list(zip(*[l[i:] for i in range(n)])))
    return result_list



0it [00:00, ?it/s]

475it [00:00, 954.33it/s]

3082it [00:00, 1342.26it/s]

5317it [00:01, 1665.29it/s]

7951it [00:01, 2316.21it/s]

10552it [00:01, 3187.19it/s]

13075it [00:01, 4319.21it/s]

15722it [00:01, 5766.98it/s]

18377it [00:01, 7536.33it/s]

20972it [00:01, 9573.91it/s]

23565it [00:01, 11808.42it/s]

26271it [00:01, 14211.21it/s]

29004it [00:02, 16600.85it/s]

31712it [00:02, 18780.11it/s]

34371it [00:02, 20592.02it/s]

37107it [00:02, 22241.31it/s]

39772it [00:02, 23402.08it/s]

42474it [00:02, 24379.52it/s]

45170it [00:02, 25098.70it/s]

47870it [00:02, 25637.50it/s]

50557it [00:02, 25901.52it/s]

53234it [00:02, 25813.41it/s]

55877it [00:03, 25746.76it/s]

58495it [00:03, 25787.77it/s]

61163it [00:03, 26047.30it/s]

63805it [00:03, 26156.77it/s]

66609it [00:03, 26694.27it/s]

69292it [00:03, 26617.07it/s]

72044it [00:03, 26880.26it/s]

74840it [00:03, 27194.67it/s]

77601it [00:03, 27314.75it/s]

80337it [00:04, 26861.45it/s]

83028it [00:04, 26816.52it/s]

8

In [ ]:
NGRAM = 2  # change this to make different N grams for each token
# now make train and valid dicts

train_data = find_ngrams(train_reviews_toked, NGRAM)
valid_data = find_ngrams(valid_reviews_toked, NGRAM)
test_data = find_ngrams(test_reviews_toked, NGRAM)

In [ ]:
# TODO: make sure to add special tokens: (n-1) <bos> at the beginning of each sentence and (n-1) <eos> at the end of each sentence
# these are needed to correctly compute n-grams with a single word etc.

In [ ]:
max_vocab_size = 100000
# save index 0 for unk and 1 for pad
PAD_IDX = 0
UNK_IDX = 1

all_train_tokens = list(mit.flatten(train_data['pos'] + train_data['neg'] + valid_data['pos'] + valid_data['neg']))
counted_tokens = Counter(all_train_tokens)

vocab, count = zip(*counted_tokens.most_common(max_vocab_size))
id2token = list(vocab)
token2id = dict(zip(vocab, range(2,2+len(vocab)))) 
id2token = ['<pad>', '<unk>'] + id2token
token2id['<pad>'] = PAD_IDX 
token2id['<unk>'] = UNK_IDX

# Lets check the dictionary by loading random token from it

random_token_id = random.randint(0, len(id2token)-1)
random_token = id2token[random_token_id]

print ("Token id {} ; token {}".format(random_token_id, id2token[random_token_id]))
print ("Token {}; token id {}".format(random_token, token2id[random_token]))

def _text2id(doc):
    return [token2id[t] if t in token2id else UNK_IDX for t in doc]

def _id2text(vec):
    return [id2token[i] for i in vec]
    
train_data_id = {}
valid_data_id = {}

train_data_id = []
for d in train_data:
    train_data_id.append(_text2id(d))
    
valid_data_id = []
for d in valid_data:
    valid_data_id.append(_text2id(d))
    
train_data_id_merged = []
valid_data_id_merged = []

for d in train_data_id:
    train_data_id_merged.append((d, 0))
    
for d in valid_data_id:
    valid_data_id_merged.append((d, 0))


In [ ]:
class ImdbDataset(Dataset):
    def __init__(self, data_list, max_inp_length=None, device='cpu'):
        """
        data_list is a list of tuples: (x,y) where x is a list of ids and y is a label
        """
        self.data = data_list
        self.max_len = max_inp_length
        self.data_tensors = []
        for (i, t) in tqdm_notebook(self.data):
            
            self.data_tensors.append((torch.LongTensor(i[:self.max_len]).to(device), torch.LongTensor([t]).to(device)))
              
    def __getitem__(self, key):
        (inp, tgt) = self.data_tensors[key]
        
        return inp, tgt, len(inp)

    def __len__(self):
        return len(self.data)

def pad(tensor, length, dim=0, pad=0):
    """Pad tensor to a specific length.
    :param tensor: vector to pad
    :param length: new length
    :param dim: (default 0) dimension to pad
    :returns: padded tensor if the tensor is shorter than length
    """
    if tensor.size(dim) < length:
        return torch.cat(
            [tensor, tensor.new(*tensor.size()[:dim],
                                length - tensor.size(dim),
                                *tensor.size()[dim + 1:]).fill_(pad)],
            dim=dim)
    else:
        return tensor
    
def batchify(batch):
    maxlen = max(batch, key = itemgetter(2))[-1]
    batch_list = []
    target_list = []
    for b in batch:
        batch_list.append(pad(b[0], maxlen, dim=0, pad=PAD_IDX))
        target_list.append(b[1])
    input_batch = torch.stack(batch_list, 0)
    target_batch = torch.stack(target_list, 0)
    
    return input_batch, target_batch

In [ ]:
train_dataset = ImdbDataset(train_data_id_merged, max_inp_length=None, device='cuda')
train_loader = DataLoader(train_dataset, batch_size=512, collate_fn=batchify, shuffle=True)

valid_dataset = ImdbDataset(valid_data_id_merged, max_inp_length=None, device='cuda')
valid_loader = DataLoader(valid_dataset, batch_size=512, collate_fn=batchify, shuffle=True)

In [16]:
out = vocab.FastText(language='en')

.vector_cache/wiki.en.vec:  19%|█▉        | 1.25G/6.60G [11:49<50:35, 1.76MB/s]


KeyboardInterrupt: 

In [ ]:
text_field.build_vocab(dataset, max_size=30000, vectors=out)

In [ ]:
# making a batch iterator
train_loader = data.BucketIterator(dataset=dataset, batch_size=4, sort_key=lambda x: len(x.reviewText), device=torch.device('cpu'), sort_within_batch=True, repeat=False)

In [ ]:
batch = next(iter(train_loader))
print(batch)

In [ ]:
def _vec2txt(vec):
    return [text_field.vocab.itos[t] for t in vec]

In [ ]:
print(batch.reviewText[0][0])
print(_vec2txt(batch.reviewText[0][0]))

# **Language Modeling**

**Goal:** compute a probabilty distribution over all possible sentences:


$p(W) = p(w_1, w_2, ..., w_T)$ - the probability that the sentence composed of words $(w_1, ..., w_T)$ in this order




This unsupervised learning problem can be framed as a sequence of supervised learning problems:


$p(W) = p(w_1) * p(w_2|w_1) * ... * p(w_T|w_1, ..., w_{T-1})$



If we have N sentences, each of them with T words / tokens, then we want to max:

$log p(W) = \sum_{n = 1}^N \sum_{i=1}^{T} log p(w_i | w_{<i})$




# **N-gram language model**

**Goal:** estimate the n-gram probabilities using counts of sequences of n consecutive words

Given a sequence of words $w$, we want to compute

  $P(w_i|w_{i−1}, w_{i−2}, …, w_{i−n+1})$

Where $w_i$ is the i-th word of the sequence.

$P(w_i|w_{i−n+1}, ..., w_{i−2}, w_{i−1}) = \frac{p(w_{i−n+1}, ..., w_{i−2}, w_{i−1}, w_i)}{\sum_{w \in V} p(w_{i−n+1}, ..., w_{i−2}, w_{i−1}, w)}$

**Key Idea:** We can estimate the probabilities using counts of n-grams in our dataset 


# Data Processing

# Bag of N-grams

# Train, Validation, Test Datasets

# Model

# Loss Function and Optimizer

# Training and Testing

# Analysis & Examples